In [1]:
from openfe import OpenFE, transform

In [2]:
# import sys
# sys.path.append('../')
import pandas as pd
from sklearn.datasets import fetch_california_housing
from openfe import OpenFE, tree_to_formula, transform
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn import preprocessing

def get_score(train_x, test_x, train_y, test_y):
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=2020)
    params = {'n_estimators': 1000, 'n_jobs': n_jobs, 'seed': 1}
    gbm = lgb.LGBMRegressor(**params)
    gbm.fit(train_x, train_y, eval_set=[(val_x, val_y)], callbacks=[lgb.early_stopping(50, verbose=False)])
    pred = pd.DataFrame(gbm.predict(test_x), index=test_x.index)
    score = mean_squared_error(test_y, pred)
    return score,test_y, pred

In [3]:
data = pd.read_excel("./数据2.0.xlsx")
columns = data.columns
# for col in columns:
#     cols = [aa for aa in columns if col in aa]
#     if len(cols) >1:
#         # 计算平均值
#         data[col + '_mean'] = data[cols].mean(axis=1)
#         # 计算方差
#         data[col + '_variance'] = data[cols].var(axis=1)
#         # 计算最大值
#         data[col + '_max'] = data[cols].max(axis=1)
#         # 计算最小值
#         data[col + '_min'] = data[cols].min(axis=1)
#         # 计算变异系数，变异系数是标准差与平均值的比值
#         data[col + '_coefficient_of_variation'] = data[cols].std(axis=1) / data[cols].mean(axis=1)

In [4]:
correlation_matrix = data.corr()
# # 选择相关系数的阈值，例如0.1
# threshold = 0.2
# # 找出低于阈值的特征
# low_correlation_features = [index for index in correlation_matrix["lattice_distortion"].index if abs(correlation_matrix["lattice_distortion"][index]) < threshold]
# # 删除这些特征
# data = data.drop(columns=low_correlation_features)
correlation_matrix.to_excel("皮尔逊相关系数矩阵-原始特征.xlsx",index = False)

In [5]:
correlation_matrix

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
X1,1.000000,0.628294,-0.081780,0.110822,0.507339,-0.167905,0.637818,0.635459,0.582331,-0.684548,-0.051633
X2,0.628294,1.000000,-0.291755,-0.521440,0.868243,-0.362547,0.869034,0.688291,0.562651,-0.837230,-0.148741
X3,-0.081780,-0.291755,1.000000,0.390102,0.122870,0.978208,-0.171351,-0.420377,-0.580441,0.194395,0.147002
X4,0.110822,-0.521440,0.390102,1.000000,-0.449718,0.433378,-0.316672,-0.273510,-0.277890,0.282114,0.203791
X5,0.507339,0.868243,0.122870,-0.449718,1.000000,0.067280,0.658028,0.369298,0.203674,-0.599247,-0.160434
X6,-0.167905,-0.362547,0.978208,0.433378,0.067280,1.000000,-0.249724,-0.497023,-0.652293,0.283118,0.121568
X7,0.637818,0.869034,-0.171351,-0.316672,0.658028,-0.249724,1.000000,0.860529,0.687883,-0.992313,-0.057392
X8,0.635459,0.688291,-0.420377,-0.273510,0.369298,-0.497023,0.860529,1.000000,0.955949,-0.895695,-0.063136
X9,0.582331,0.562651,-0.580441,-0.277890,0.203674,-0.652293,0.687883,0.955949,1.000000,-0.741366,-0.066129
X10,-0.684548,-0.837230,0.194395,0.282114,-0.599247,0.283118,-0.992313,-0.895695,-0.741366,1.000000,0.050263


In [6]:
# data.to_excel("初步筛选后的特征.xlsx",index = False)

In [7]:
def data_generator(f_path, seed=88): 
    df_all = pd.read_excel(f_path)
    #create a min max processing object
    composition = df_all
    scaler = preprocessing.StandardScaler().fit(composition)
    normalized_composition = scaler.transform(composition)
    return normalized_composition,scaler

In [8]:
data_trans,scaler = data_generator("./数据2.0.xlsx", seed=42)
data = pd.DataFrame(data_trans,columns=columns)
data.to_excel("归一化后的数据.xlsx",index = False)

In [9]:
# data = pd.read_excel("./原始数据2.xlsx")

In [10]:
aa = data.copy(deep = True)

In [11]:
if __name__ == '__main__':
    n_jobs = 4
#     data = fetch_california_housing(as_frame=True).frame
#     label = data[['MedHouseVal']]
#     del data['MedHouseVal']
#     label = -np.log10(1-data[["average_coulombic_efficiency"]]/100)
    label=data[["Y"]]
    del data["Y"]
    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=1)
    # get baseline score
    score,_,_ = get_score(train_x, test_x, train_y, test_y)
    print("The MSE before feature generation is", score)
    # feature generation
    ofe = OpenFE()
    ofe.fit(data=train_x, label=train_y, n_jobs=n_jobs,n_data_blocks = 1)
    # OpenFE recommends a list of new features. We include the top 10
    # generated features to see how they influence the model performance
    train_x, test_x = transform(train_x, test_x, ofe.new_features_list[:10], n_jobs=n_jobs)
    score,test_y, pred = get_score(train_x, test_x, train_y, test_y)
    print("The MSE after feature generation is", score)
    print("The top 10 generated features are")
    for feature in ofe.new_features_list[:10]:
        print(tree_to_formula(feature))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 212
[LightGBM] [Info] Number of data points in the train set: 75, number of used features: 10
[LightGBM] [Info] Start training from score -0.041404
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:05<00:00,  3.17it/s]


443 same features have been deleted.
The number of remaining candidate features is 305
Start stage II selection.


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.75it/s]


Finish data processing.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000685 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6235
[LightGBM] [Info] Number of data points in the train set: 75, number of used features: 314
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

In [12]:
new_feature_name = []
origin_data = pd.read_excel("./数据2.0.xlsx")
for i in list(origin_data.columns)[:-1]:
    new_feature_name.append(i)

In [13]:
for feature in ofe.new_features_list[:10]:
    new_feature_name.append(tree_to_formula(feature))
new_feature_name.append(list(origin_data.columns)[-1])

In [14]:
train_data = pd.concat([train_x,train_y],axis=1)
test_data = pd.concat([test_x,test_y],axis=1)

In [15]:
data_concat = pd.concat([train_data,test_data],axis=0)
data_concat.columns = new_feature_name
data_concat.to_excel("高价值特征生成.xlsx",index=False)

# 贪心算法特征选择

In [16]:
import numpy as np
import pandas as pd

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [18]:
def Liner_model(datasets,cols):
    skf = KFold(n_splits=5, shuffle=True,random_state=888)
    valid_ssr =[]
    x = datasets[:,cols]
    y = datasets[:,-1:]
    for i, (trn_idx, val_idx) in enumerate(skf.split(x, y)):
        trn_x, trn_y = x[trn_idx], y[trn_idx]
        val_x, val_y = x[val_idx], y[val_idx]
        # 创建并拟合线性回归模型
        model = LinearRegression()
        model.fit(trn_x, trn_y)
        # 获得残差和
        residual_sum_of_squares = np.sum((model.predict(val_x) - val_y) ** 2)
        valid_ssr.append(residual_sum_of_squares)
    #返回当前特征组合的交叉验证的残差和
    return np.array(valid_ssr).mean()

In [19]:
def bic(data, features):
    # 估计线性回归模型的参数和残差平方和
    rss = Liner_model(data,features)
    # 计算BIC值
    n = len(data)*0.8
    k = len(features)
    
    return n * np.log(rss / n) + k * np.log(n),rss

In [20]:
def get_best_features(data):
    # 初始化特征集合为空
    best_features = []
    current_best_bic = []
    current_best_feature = []
    SSE = []
    for a in range(data.shape[1] - 1):
        # 初始化最小的BIC值为无穷大
        Bics = []
        cols = []
        sses = []
        # 遍历所有的特征
        for i in range(data.shape[1] - 1):
            col = current_best_feature.copy()
            if i not in col:
                col.append(i)
                # 计算添加后的特征子集的BIC值
                new_bic,sse = bic(data, col)
                Bics.append(new_bic)
                cols.append(col)
                sses.append(sse)
        current_best_feature = cols[np.array(sses).argmin()].copy()
        current_best_bic.append(Bics[np.array(Bics).argmin()])
        best_features.append(current_best_feature)
        SSE.append(sses[np.array(Bics).argmin()])
    return best_features,SSE

In [21]:
import statsmodels.api as sm
import pandas as pd
def get_bis(features,path):
    org_data = pd.read_excel(path).fillna(0)
    columns = list(org_data.columns)
    data_y = org_data[[columns[i] for i in features]]
    model_1 = sm.OLS(org_data['Y'], sm.add_constant(data_y)).fit()
    return model_1.bic

In [22]:
def bic_choose_feature(best_features,path):
    bics = []
    for i in range(len(best_features)):
        bic = get_bis(best_features[i],path)
        bics.append(bic)
    min_index = np.array(bics).argmin()
    return best_features[min_index],bics

In [23]:
path = "高价值特征生成.xlsx"
data = pd.read_excel(path).fillna(0)

In [24]:
best_features,SSE = get_best_features(np.array(data))
pd.DataFrame(best_features).to_excel("best_feature_candinate.xlsx")
bic_feature,bics = bic_choose_feature(best_features,path)

In [25]:
# bic_feature = best_features[4]

In [26]:
sse_bics = pd.concat((pd.DataFrame(SSE),pd.DataFrame(bics)),axis=1)
sse_bics.columns = ["SSE","BIC"]
sse_bics.to_excel("SSE_and_BIC.xlsx",index = False)

In [27]:
org_data = pd.read_excel("高价值特征生成.xlsx").fillna(0)
columns = list(org_data.columns)
filter_data = org_data[[columns[i] for i in bic_feature]]

In [28]:
data3 = pd.concat((filter_data,org_data.iloc[:,-1:]),axis = 1)
data3.to_excel("BIC筛选后特征数据集.xlsx",index = False)

In [29]:
data3.corr().to_excel("皮尔逊相关系数矩阵-特征筛选后.xlsx",index = False)

In [30]:
data3.corr()

,"GroupByThenMax(X6,X4)",residual(X6),"GroupByThenStd(X6,X4)","max(X3,X8)",(X2/X3),Y
"GroupByThenMax(X6,X4)",1.000000,0.026925,0.009732,0.025857,-0.087356,0.448627
residual(X6),0.026925,1.000000,-0.061780,0.027331,0.004459,0.178645
"GroupByThenStd(X6,X4)",0.009732,-0.061780,1.000000,-0.111829,-0.009590,-0.161281
"max(X3,X8)",0.025857,0.027331,-0.111829,1.000000,0.017950,0.147827
(X2/X3),-0.087356,0.004459,-0.009590,0.017950,1.000000,-0.176945
Y,0.448627,0.178645,-0.161281,0.147827,-0.176945,1.000000


# 比较不同模型的效果

In [31]:
#线性回归

In [32]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import sklearn.metrics as metrics

def Liner_model(datasets):
    skf = KFold(n_splits=5, shuffle=True,random_state=888)
    intercepts =[]
    coefficients = []
    x = datasets[:,:-1]
    y = datasets[:,-1:]
    val_ys = []
    pre_ys = []
    r2s = []
    for i, (trn_idx, val_idx) in enumerate(skf.split(x, y)):
        trn_x, trn_y = x[trn_idx], y[trn_idx]
        val_x, val_y = x[val_idx], y[val_idx]
        # 创建并拟合线性回归模型
        model = LinearRegression()
        model.fit(trn_x, trn_y)
        intercept = model.intercept_
        # 获取参数
        coefficient = model.coef_
        intercepts.append(intercept)
        coefficients.append(coefficient)
        # 计算MAE，MSE和R2
#         xgb_mae = metrics.mean_absolute_error(xgb_val, xgb_pre)
#         xgb_mse = metrics.mean_squared_error(xgb_val, xgb_pre)
        line_r2 = metrics.r2_score(val_y, model.predict(val_x))
        r2s.append(line_r2)
        pre_ys.append(model.predict(val_x))
        val_ys.append(val_y)
    return intercepts,coefficients,r2s,pre_ys,val_ys

In [75]:
path = "BIC筛选后特征数据集.xlsx"
data = pd.read_excel(path).fillna(0)

In [76]:
data

,"GroupByThenMax(X6,X4)",residual(X6),"GroupByThenStd(X6,X4)","max(X3,X8)",(X2/X3),Y
0,2.242085,0.408148,0.820746,1.418671,-0.530078,1.795592
1,1.804372,0.643187,0.842607,0.688715,0.423913,0.165822
2,2.038787,0.754382,0.986674,-0.284980,-0.202701,1.111060
3,1.625449,0.099676,0.853398,-0.102070,-2.860342,0.180828
4,2.038787,0.446217,0.986674,0.585284,2.164041,-0.747228
...,...,...,...,...,...,...
113,1.601074,0.601074,1.049604,1.601160,0.182340,-0.910194
114,2.038787,0.809355,0.986674,0.771165,-0.186913,-1.479930
115,2.038787,0.877602,0.986674,0.871204,-1.462329,0.336394
116,1.187735,0.315631,0.919892,1.531590,-0.483438,-0.138087


In [33]:
path = "BIC筛选后特征数据集.xlsx"
data = pd.read_excel(path).fillna(0)
data["Y"] = data["Y"] * scaler.scale_[-1] + scaler.mean_[-1] 
intercepts,coefficients,r2s,pre_ys,val_ys = Liner_model(np.array(data))

# 导入机器学习包
import sklearn.metrics as metrics
line_val = [a for i in val_ys for a in i]
line_pre = [a for i in pre_ys for a in i]

# 计算MAE，MSE和R2
line_mae = metrics.mean_absolute_error(line_val, line_pre)
line_mse = metrics.mean_squared_error(line_val, line_pre)
line_r2 = metrics.r2_score(line_val, line_pre)

# 打印结果
print("line_MAE:", line_mae)
print("line_MSE:", line_mse)
print("line_R2:", line_r2)


line_pre_data = pd.concat((pd.DataFrame(line_val),pd.DataFrame(line_pre)),axis = 1)
line_pre_data.columns = ["line_Actual","line_Predict"]
line_pre_data.to_excel("line_pre_data.xlsx",index =False)

line_MAE: 0.3229177086378618
line_MSE: 0.16329042392757473
line_R2: 0.255858910893054


In [34]:
#随机森林

In [35]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import optuna

D:\Software\Anaconda3\envs\matterai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
def read_data(f_path):
    if f_path.endswith(".csv"):
        data = pd.read_csv(f_path)
    elif f_path.endswith(".xls") or f_path.endswith(".xlsx"):
        data = pd.read_excel(f_path)
    return data

In [37]:
def RF_train(params,data,Predict = False,Output = False):
    dataset = np.array(data)
    x = dataset[:,:-1]
    y = dataset[:,-1:].reshape(-1)
    n_splits = 5
    skf = KFold(n_splits=n_splits, shuffle=True, random_state=2020)
    min_mae = []
    pre = 0
    val_ys = []
    pre_ys = []
    for i, (trn_idx, val_idx) in enumerate(skf.split(x, y)):
        trn_x, trn_y = x[trn_idx], y[trn_idx]
        val_x, val_y = x[val_idx], y[val_idx]
        rf_model = RandomForestRegressor(n_estimators=params["n_estimators"],max_depth=params["max_depth"])
        rf_model.fit(trn_x,trn_y)
        val_ys.append(val_y)
        pre_ys.append(rf_model.predict(val_x))
        if Predict is True:
            pre += rf_model.predict(test)/n_splits
        else:
#                 min_r2.append(rf_model.score(val_x,val_y))
            val_mae = abs(abs(rf_model.predict(val_x))-abs(val_y)).mean()
            min_mae.append(val_mae)
    if Predict is True:
        return pre
    elif Output is True:
        return val_ys,pre_ys
    else:     
        return np.array(min_mae).mean()

In [38]:
def rf_objective(trial,data):

    params = {
          'n_estimators': trial.suggest_int("n_estimators", 10, 100,5),
          'max_depth': trial.suggest_int("max_depth", 1,10,1),
          }

    loss = RF_train(params,data)

    return loss

In [39]:
def optimizer_optuna(n_trials, algo,optuna_objective,data):

    #定义使用TPE或者GP
    if algo == "TPE":
        algo = optuna.samplers.TPESampler(n_startup_trials = 10, n_ei_candidates = 24)
    elif algo == "GP":
        from optuna.integration import SkoptSampler
        import skopt
        algo = SkoptSampler(skopt_kwargs={'base_estimator':'GP', #选择高斯过程
                                          'n_initial_points':10, #初始观测点10个
                                          'acq_func':'EI'} #选择的采集函数为EI，期望增量
                           )

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = algo #要使用的具体算法
                                , direction="minimize" #优化的方向，可以填写minimize或maximize
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(lambda trial: optuna_objective(trial, data) #目标函数
                   , n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   , show_progress_bar=True #要不要展示进度条呀？
                  )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")

    return study.best_trial.params, study.best_trial.values

In [69]:
path = "./BIC筛选后特征数据集.xlsx"

In [70]:
data = read_data(path)

In [42]:
rf_best_params,rf_best_mae = optimizer_optuna(n_trials = 100, algo="TPE", optuna_objective=rf_objective, data=data)

[I 2024-05-28 22:58:19,733] A new study created in memory with name: no-name-91268712-2c9c-446c-b642-f8661fcb2ba5
Best trial: 0. Best value: 0.551274:   2%|▉                                            | 2/100 [00:00<00:20,  4.88it/s]

[I 2024-05-28 22:58:20,003] Trial 0 finished with value: 0.551273619399501 and parameters: {'n_estimators': 35, 'max_depth': 2}. Best is trial 0 with value: 0.551273619399501.
[I 2024-05-28 22:58:20,162] Trial 1 finished with value: 0.5946299381318157 and parameters: {'n_estimators': 20, 'max_depth': 1}. Best is trial 0 with value: 0.551273619399501.


Best trial: 2. Best value: 0.531676:   3%|█▎                                           | 3/100 [00:01<00:42,  2.26it/s]

[I 2024-05-28 22:58:20,884] Trial 2 finished with value: 0.5316756897756789 and parameters: {'n_estimators': 85, 'max_depth': 10}. Best is trial 2 with value: 0.5316756897756789.


Best trial: 2. Best value: 0.531676:   4%|█▊                                           | 4/100 [00:01<00:40,  2.39it/s]

...

[I 2024-05-28 22:59:14,745] Trial 97 finished with value: 0.51440097525642 and parameters: {'n_estimators': 75, 'max_depth': 8}. Best is trial 28 with value: 0.4983475248425112.


Best trial: 28. Best value: 0.498348:  99%|██████████████████████████████████████████▌| 99/100 [00:55<00:00,  1.52it/s]

[I 2024-05-28 22:59:15,449] Trial 98 finished with value: 0.5241125065818288 and parameters: {'n_estimators': 80, 'max_depth': 10}. Best is trial 28 with value: 0.4983475248425112.


Best trial: 28. Best value: 0.498348: 100%|██████████████████████████████████████████| 100/100 [00:56<00:00,  1.77it/s]

[I 2024-05-28 22:59:16,182] Trial 99 finished with value: 0.6015875376544945 and parameters: {'n_estimators': 100, 'max_depth': 1}. Best is trial 28 with value: 0.4983475248425112.

 
 best params:  {'n_estimators': 65, 'max_depth': 6} 
 
 best score:  [0.4983475248425112] 



In [43]:
val_ys,pre_ys= RF_train(params=rf_best_params,data=data,Output=True)

In [44]:
# 导入机器学习包
import sklearn.metrics as metrics
rf_val = [a for i in val_ys for a in i]
rf_pre = [a for i in pre_ys for a in i]

rf_val = np.array(rf_val) * scaler.scale_[-1] + scaler.mean_[-1] 
rf_pre = np.array(rf_pre) * scaler.scale_[-1] + scaler.mean_[-1] 
# 计算MAE，MSE和R2
rf_mae = metrics.mean_absolute_error(rf_val, rf_pre)
rf_mse = metrics.mean_squared_error(rf_val, rf_pre)
rf_r2 = metrics.r2_score(rf_val, rf_pre)

# 打印结果
print("RF_MAE:", rf_mae)
print("RF_MSE:", rf_mse)
print("RF_R2:", rf_r2)

RF_MAE: 0.3115652916114944
RF_MSE: 0.16218628418559444
RF_R2: 0.26089065574594184


In [45]:
RF_pre_data = pd.concat((pd.DataFrame(rf_val),pd.DataFrame(rf_pre)),axis = 1)
RF_pre_data.columns = ["RF_Actual","RF_Predict"]
RF_pre_data.to_excel("RF_pre_data.xlsx",index =False)

# XGBOOST

In [46]:
import xgboost as xgb

In [47]:
def xgb_train(params, data, data2=None, Predict=False, Output=False):
    dataset = np.array(data)
    x = dataset[:, :-1]
    y = dataset[:, -1:]
    if Predict is True:
        test_x = np.array(data2)
    n_splits = 5
    skf = KFold(n_splits=n_splits, shuffle=True, random_state=2020)
    min_mae = []
    pre = 0
    
    val_ys = []
    pre_ys = []
    evals_result = {}
    for i, (trn_idx, val_idx) in enumerate(skf.split(x, y)):
        trn_x, trn_y = x[trn_idx], y[trn_idx]
        val_x, val_y = x[val_idx], y[val_idx]
        dataTrain = xgb.DMatrix(trn_x, trn_y)
        dataVal = xgb.DMatrix(val_x, val_y)
        watchlist = [(dataVal, 'test'), (dataTrain, 'train')]
        if Predict is True:
            dataTest = xgb.DMatrix(test_x)
            bst = xgb.train(params=params, dtrain=dataTrain, num_boost_round=5000, evals=watchlist,
                            callbacks=[xgb.callback.EarlyStopping(50)])
            pre += bst.predict(dataTest) / n_splits

        
        elif Output is True:
            bst = xgb.train(params=params, dtrain=dataTrain, num_boost_round=5000, evals=watchlist,
                            callbacks=[xgb.callback.EarlyStopping(50)],verbose_eval=False)
            val_ys.append(val_y)
            pre_ys.append(bst.predict(xgb.DMatrix(val_x)))
            

        else:
            bst = xgb.train(params=params, dtrain=dataTrain, num_boost_round=5000, evals=watchlist,
                            evals_result=evals_result, callbacks=[xgb.callback.EarlyStopping(50)],verbose_eval=False)
            min_mae.append(min(list(evals_result["test"].values())[0]))


    if Predict is True:
        return pre
    elif Output is True:
        return val_ys, pre_ys
    else:
        return np.array(min_mae).mean()
    
    
    
def xgb_objective(trial, data):
    params = {
        'subsample': trial.suggest_float("subsample", 0.2, 0.8),
        'eta': trial.suggest_float("eta", 0.01, 0.1, step=0.01),
        'max_depth': trial.suggest_int("max_depth", 1, 5, 1),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.2, 0.8, step=0.1),
        'gamma': trial.suggest_float("gamma", 0, 1),
        'eval_metric': "mae",
        "verbose":0
    }
    loss = xgb_train(params, data)

    return loss

In [48]:
def get_prediction(params, data1, data2):
    pre_reslust = xgb_train(params=params, data=data1, data2=data2, Predict=True)
    return pre_reslust

In [49]:
file_path = "./BIC筛选后特征数据集.xlsx"
data = read_data(file_path)

In [50]:
xgb_best_params, xgb_best_mae = optimizer_optuna(n_trials=100, algo="TPE", optuna_objective=xgb_objective,
                                               data=data)

[I 2024-05-28 22:59:16,974] A new study created in memory with name: no-name-f9076e55-8515-4190-9a0e-bfd6ee0c9015
Best trial: 0. Best value: 0.609565:   1%|▍                                            | 1/100 [00:09<16:09,  9.79s/it]

[I 2024-05-28 22:59:26,763] Trial 0 finished with value: 0.6095653635058282 and parameters: {'subsample': 0.5278271063863788, 'eta': 0.04, 'max_depth': 1, 'colsample_bytree': 0.30000000000000004, 'gamma': 0.7678297800649218}. Best is trial 0 with value: 0.6095653635058282.


Best trial: 1. Best value: 0.607552:   2%|▉                                            | 2/100 [00:32<28:09, 17.24s/it]

[I 2024-05-28 22:59:49,224] Trial 1 finished with value: 0.6075521642018271 and parameters: {'subsample': 0.26429177120555, 'eta': 0.02, 'max_depth': 1, 'colsample_bytree': 0.5, 'gamma': 0.4162526004084376}. Best is trial 1 with value: 0.6075521642018271.


Best trial: 1. Best value: 0.607552:   3%|█▎                                           | 3/100 [00:41<21:48, 13.49s/it]

...

[I 2024-05-28 23:11:37,692] Trial 97 finished with value: 0.5795458015985787 and parameters: {'subsample': 0.3103065001431479, 'eta': 0.09, 'max_depth': 3, 'colsample_bytree': 0.4, 'gamma': 0.9208579043570202}. Best is trial 57 with value: 0.56351861745605.


Best trial: 57. Best value: 0.563519:  99%|██████████████████████████████████████████▌| 99/100 [12:34<00:06,  6.61s/it]

[I 2024-05-28 23:11:51,233] Trial 98 finished with value: 0.5947018034614461 and parameters: {'subsample': 0.30864166677523186, 'eta': 0.01, 'max_depth': 3, 'colsample_bytree': 0.4, 'gamma': 0.9265111528643146}. Best is trial 57 with value: 0.56351861745605.


Best trial: 57. Best value: 0.563519: 100%|██████████████████████████████████████████| 100/100 [12:37<00:00,  7.57s/it]

[I 2024-05-28 23:11:54,201] Trial 99 finished with value: 0.6013234316830294 and parameters: {'subsample': 0.2774026929378692, 'eta': 0.09, 'max_depth': 3, 'colsample_bytree': 0.30000000000000004, 'gamma': 0.9003852387014646}. Best is trial 57 with value: 0.56351861745605.

 
 best params:  {'subsample': 0.3310132690432152, 'eta': 0.09999999999999999, 'max_depth': 3, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.8572399092445809} 
 
 best score:  [0.56351861745605] 



In [51]:
val_ys, pre_ys = xgb_train(xgb_best_params, data, data2=None, Predict=False, Output=True)

In [52]:
# 导入机器学习包
import sklearn.metrics as metrics
xgb_val = [a for i in val_ys for a in i]
xgb_pre = [a for i in pre_ys for a in i]
xgb_val = np.array(xgb_val) * scaler.scale_[-1] + scaler.mean_[-1] 
xgb_pre = np.array(xgb_pre) * scaler.scale_[-1] + scaler.mean_[-1] 
# 计算MAE，MSE和R2
xgb_mae = metrics.mean_absolute_error(xgb_val, xgb_pre)
xgb_mse = metrics.mean_squared_error(xgb_val, xgb_pre)
xgb_r2 = metrics.r2_score(xgb_val, xgb_pre)

# 打印结果
print("XGB_MAE:", xgb_mae)
print("XGB_MSE:", xgb_mse)
print("XGB_R2:", xgb_r2)

XGB_MAE: 0.30366529795027675
XGB_MSE: 0.15263435588997473
XGB_R2: 0.30442035059275396


In [53]:
XGB_pre_data = pd.concat((pd.DataFrame(xgb_val),pd.DataFrame(xgb_pre)),axis = 1)
XGB_pre_data.columns = ["XGB_Actual","XGB_Predict"]
XGB_pre_data.to_excel("XGB_pre_data.xlsx",index =False)

# 神经网络

In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import pandas as pd
import copy
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 搭建全连接神经网络回归
class MLPregression(nn.Module):
    def __init__(self, params, data):
        super(MLPregression, self).__init__()
        self.layber_number = params["layber_number"]
        self.unit = params["unit"]
        # 第一个隐含层
        self.hidden1 = nn.Linear(in_features=data.shape[-1]-1, out_features=self.unit, bias=True)
        # 第二个隐含层
        self.hidden2 = nn.Linear(self.unit, self.unit)
        # 第三个隐含层
        self.hidden3 = nn.Linear(128, 256)
        # 回归预测层
        self.hidden5 = nn.Linear(self.unit, 64)
        self.predict = nn.Linear(64, 1)
        self.relu = nn.functional.relu
        self.dropout = nn.Dropout(params["drop_out"])

    # 定义网络前向传播路径
    def forward(self, x):
        x = self.relu(self.hidden1(x))
        for i in range(self.layber_number):
            x = self.dropout(self.relu(self.hidden2(x)))
        x = self.dropout(self.relu(self.hidden5(x)))
        output = self.predict(x)
        # 输出一个一维向量
        return output[:, 0]

In [79]:
class FastTensorDataLoader:
    """
    A DataLoader-like object for a set of tensors that can be much faster than
    TensorDataset + DataLoader because dataloader grabs individual indices of
    the dataset and calls cat (slow).
    Source: https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
    """
    def __init__(self, *tensors, batch_size=32, shuffle=False):
        """
        Initialize a FastTensorDataLoader.
        :param *tensors: tensors to store. Must have the same length @ dim 0.
        :param batch_size: batch size to load.
        :param shuffle: if True, shuffle the data *in-place* whenever an
            iterator is created out of this object.
        :returns: A FastTensorDataLoader.
        """
        assert all(t.shape[0] == tensors[0].shape[0] for t in tensors)
        self.tensors = tensors

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches
    def __iter__(self):
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = [t[r] for t in self.tensors]
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.dataset_len:
            raise StopIteration
        batch = tuple(t[self.i:self.i+self.batch_size] for t in self.tensors)
        self.i += self.batch_size
        return batch

    def __len__(self):
        return self.n_batches

In [80]:
def train_model(params, model, dataloaders, criterion, optimizer, num_epochs, i, savebest=False,Output=False):
    since = time.time()
    best_loss = 10000000
    model.to(device)
    train_losses = []
    valid_losses = []

    for epoch in range(num_epochs):

        # 训练和验证
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # 训练
            else:
                model.eval()  # 验证

            running_loss = 0.0
            running_corrects = 0

            # 把数据都取个遍
            for inputs, labels in dataloaders[phase]:

                inputs = inputs.to(device)
                labels = labels.to(device)
                # 清零
                optimizer.zero_grad()
                # 只有训练的时候计算和更新梯度

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #                     print(outputs.shape, labels.shape)
                    loss = criterion(outputs, labels)
                    #                     print("loss为：",loss)
                    # 训练阶段更新权重
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                #                 计算损失

                running_loss += loss.item() * inputs.size(0)
            epoch_loss = running_loss / dataloaders[phase].dataset_len
            #             print(epoch_loss)
            time_elapsed = time.time() - since
            #             print('Time elapsed {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            #             print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # 得到最好那次的模型
            if phase == 'valid' and epoch_loss < best_loss:
                best_loss = epoch_loss
#                 print(best_loss)
#                 print(outputs[0],labels[0])
                if savebest is True:
                    best_model_wts = copy.deepcopy(model.state_dict())
                    state = {
                        'state_dict': model.state_dict(),  # 字典里key就是各层的名字，值就是训练好的权重
                        'best_loss': best_loss,
                        'optimizer': optimizer.state_dict(),  # 优化器的状态信息
                    }
                    filename = './NN_best' + str(i) + '.pth'
                    torch.save(state, filename)
                    inputs_list = []
                    labels_list = []
                    for inputs, labels in dataloaders[phase]:
                        inputs = inputs.to(device)
                        labels = labels.to(device)
                        inputs_list.append(inputs)
                        labels_list.append(labels)
                    final_inputs = torch.cat(inputs_list, dim=0)
                    final_labels = torch.cat(labels_list, dim=0)
                    pre_y = model(final_inputs)
                    val_y = final_labels

                    
            if phase == 'valid':
                valid_losses.append(epoch_loss)
            #                 scheduler.step(epoch_loss)#学习率衰减
            if phase == 'train':
                train_losses.append(epoch_loss)

    if savebest is True:
        return train_losses, valid_losses, best_loss, pre_y,val_y
    else:
        return best_loss

In [81]:
def model_cross_train(params, data, savebest=False):
    x = data[:, :-1]
    y = data[:, -1:]
    loss = []
    skf = KFold(n_splits=5, shuffle=True, random_state=42)
    train_losses = []
    valid_losses = []
    val_ys = []
    pre_ys = []
    for i, (trn_idx, val_idx) in enumerate(skf.split(x, y)):
        trn_x, trn_y = x[trn_idx], y[trn_idx]
        val_x, val_y = x[val_idx], y[val_idx]
#         print(f"val is {len(val_y)}")
        # 将数据集转为张量
        X_train_t = torch.from_numpy(trn_x.astype(np.float32))
        y_train_t = torch.from_numpy(trn_y.squeeze().astype(np.float32))
        X_valid_t = torch.from_numpy(val_x.astype(np.float32))
        y_valid_t = torch.from_numpy(val_y.squeeze().astype(np.float32))

        # 将训练数据处理为数据加载器
        #         train_data = Data.TensorDataset(X_train_t, y_train_t)
        #         valid_data = Data.TensorDataset(X_valid_t, y_valid_t)

        #         train_loader = Data.DataLoader(dataset = train_data, batch_size = params['batch_size'],
        #                                        shuffle = True, num_workers = 1)
        #         val_loader = Data.DataLoader(dataset = valid_data, batch_size = params['batch_size'],
        #                                        shuffle = True, num_workers = 1)
        train_loader = FastTensorDataLoader(X_train_t, y_train_t, batch_size=params['batch_size'],
                                            shuffle=True)
        val_loader = FastTensorDataLoader(X_valid_t, y_valid_t, batch_size=params['batch_size'],
                                          shuffle=True)
        # 实例化
        model = MLPregression(params, data)
        # 损失函数
        criterion = torch.nn.L1Loss()
        # 优化器设置
        optimizer_ft = optim.Adam(model.parameters(), lr=params["lr"])
        # scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)#学习率每7个epoch衰减成原来的1/1
        # 数据加载器
        dataloaders = {'train': train_loader, 'valid': val_loader}
        
        if savebest is True:
            train_loss, valid_loss, best_loss, pre_y,act_y = train_model(params, model, dataloaders, criterion, optimizer_ft,
                                                            num_epochs=100, i=i, savebest=True)
            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
            loss.append(best_loss)
            val_ys.append(act_y)
            pre_ys.append(pre_y)
        else:
            best_loss = train_model(params, model, dataloaders, criterion, optimizer_ft, num_epochs=100, i=i)
            loss.append(best_loss)

    if savebest is True:
        return train_losses, valid_losses, loss,val_ys,pre_ys
    else:
        return np.array(loss).mean()


def model_objective(trial, data):
    params = {
          'lr': trial.suggest_float("lr", 1e-5, 1e-2),
          'batch_size': trial.suggest_int("batch_size", 16,128,16),
          'drop_out':trial.suggest_float("drop_out", 0, 0.2),
          'unit': trial.suggest_int("unit", 16, 128,16),
          'layber_number':trial.suggest_int("layber_number", 1, 16,1)
          }
    loss = model_cross_train(params, data)
    return loss

In [82]:
def data_generator(f_path, seed=42): 
    df_all = pd.read_excel(f_path)
    #create a min max processing object
    composition = df_all.iloc[:,:-1]
    scaler = preprocessing.StandardScaler().fit(composition)
    normalized_composition = scaler.transform(composition)
    
    return normalized_composition

In [72]:
file_path = "./BIC筛选后特征数据集.xlsx"
# file_path = "./原始数据2.xlsx"
test_x = data_generator(file_path, seed=42)
label = np.array(pd.read_excel(file_path)["Y"]).reshape(-1,1)
data =np.hstack((test_x,label))

In [83]:
# file_path = "./BIC筛选后特征数据集.xlsx"
# data = pd.read_excel(path).fillna(0)

In [84]:
data = np.array(aa)

In [85]:
nn_best_params, nn_best_mae = optimizer_optuna(n_trials=100, algo="TPE", optuna_objective=model_objective,
                                               data=data)

[I 2024-05-29 10:29:15,011] A new study created in memory with name: no-name-34eacffe-3c07-467d-888d-5ad47e706702
Best trial: 0. Best value: 0.708384:   1%|▍                                            | 1/100 [00:04<07:18,  4.42s/it]

[I 2024-05-29 10:29:19,435] Trial 0 finished with value: 0.7083837389945984 and parameters: {'lr': 0.00618714213928684, 'batch_size': 96, 'drop_out': 0.007483762696852559, 'unit': 112, 'layber_number': 5}. Best is trial 0 with value: 0.7083837389945984.


Best trial: 1. Best value: 0.69061:   2%|▉                                             | 2/100 [00:10<08:26,  5.17s/it]

[I 2024-05-29 10:29:25,119] Trial 1 finished with value: 0.690609872341156 and parameters: {'lr': 0.004822170403328864, 'batch_size': 48, 'drop_out': 0.17207605315118482, 'unit': 32, 'layber_number': 1}. Best is trial 1 with value: 0.690609872341156.


Best trial: 1. Best value: 0.69061:   3%|█▍                                            | 3/100 [00:16<09:24,  5.81s/it]

...

[I 2024-05-29 10:42:11,047] Trial 97 finished with value: 0.6600636184215546 and parameters: {'lr': 0.008398096337017933, 'batch_size': 112, 'drop_out': 0.09388344008841629, 'unit': 128, 'layber_number': 7}. Best is trial 71 with value: 0.6560588479042053.


Best trial: 71. Best value: 0.656059:  99%|██████████████████████████████████████████▌| 99/100 [13:01<00:06,  6.59s/it]

[I 2024-05-29 10:42:16,239] Trial 98 finished with value: 0.686643648147583 and parameters: {'lr': 0.008332728406487033, 'batch_size': 128, 'drop_out': 0.07384874693637226, 'unit': 80, 'layber_number': 9}. Best is trial 71 with value: 0.6560588479042053.


Best trial: 71. Best value: 0.656059: 100%|██████████████████████████████████████████| 100/100 [13:05<00:00,  7.86s/it]

[I 2024-05-29 10:42:20,858] Trial 99 finished with value: 0.6904806315898895 and parameters: {'lr': 0.008055101481683176, 'batch_size': 112, 'drop_out': 0.08602630032654668, 'unit': 64, 'layber_number': 7}. Best is trial 71 with value: 0.6560588479042053.

 
 best params:  {'lr': 0.008190696772301592, 'batch_size': 80, 'drop_out': 0.09486115704544952, 'unit': 128, 'layber_number': 8} 
 
 best score:  [0.6560588479042053] 



In [86]:
train_losses, valid_losses, loss,val_ys,pre_ys = model_cross_train(nn_best_params, data, savebest=True)

In [87]:
criterion = torch.nn.L1Loss()
criterion(pre_ys[1].cpu(),val_ys[1].cpu())
# 导入机器学习包
import sklearn.metrics as metrics
nn_val = [a for i in val_ys for a in i.cpu().data.numpy()]
nn_pre = [a for i in pre_ys for a in i.cpu().data.numpy()]

nn_val = np.array(nn_val) * scaler.scale_[-1] + scaler.mean_[-1] 
nn_pre = np.array(nn_pre) * scaler.scale_[-1] + scaler.mean_[-1] 
# 计算MAE，MSE和R2
nn_mae = metrics.mean_absolute_error(nn_val, nn_pre)
nn_mse = metrics.mean_squared_error(nn_val, nn_pre)
nn_r2 = metrics.r2_score(nn_val, nn_pre)

# 打印结果
print("NN_MAE:", nn_mae)
print("NN_MSE:", nn_mse)
print("NN_R2:", nn_r2)

NN_MAE: 0.32478404
NN_MSE: 0.17659202
NN_R2: 0.19524130964784248


In [64]:
NN_pre_data = pd.concat((pd.DataFrame(nn_val),pd.DataFrame(nn_pre)),axis = 1)
NN_pre_data.columns = ["NN_Actual","NN_Predict"]
NN_pre_data.to_excel("NN_pre_data.xlsx",index =False)

In [65]:
len(nn_val)

118